# Extração e seleção de características

As características que são extraídas dos dados dependem de sua natureza. Os dados EMG são sinais elétricos coletados dentro de um período de tempo, portanto são dados no domínio do tempo. As características deste domínio são extraídas dele. Também é possível transformar os dados EMG para o domínio da frequência e extrair características neste domínio. Existem diversas características que podem ser extraídas de cada domínio, entretando nem todas elas serão relevantes. Cada problema se beneficia de características ou combinações delas. Portanto, é preciso que haja uma seleção de características para encontrar a combinação de características que trará melhor resultado na posterior classificação dos dados.

## Reutilizando os passos anteriores

É necessário carregar os dados pré-processados, para dar início à extração de características. No jupyter notebook podemos utilizar o namespace completo de outro notebook:

In [1]:
%run preprocessing.ipynb

(20000, 2)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 1
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 2
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 3
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 4
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 5
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 6
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 7
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em EMG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) \\$
    $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn([x_i - threshold][x_{i+1} - threshold]) \\$
    $ sgn(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $
    $Sugestão: threshold = 0.4$

### Domínio da frequência

1. Auto Regressive (AR)

    > $ - \sum_{j=1}^{\rho}\alpha_j x_{j-1} + w_n $

2. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

3. Mean Frequency (FMN)

    > $ \sum_{j=1}^{M}f_j PSD_j \Big{ / } \sum_{j=1}^{M}PSD_j $

4. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{i=1}^{M}|w|_i $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Desafio 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

### Extraindo características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características VAR & RMS (domínio do tempo) e FDM & MMDF (domínio da frequência).

In [2]:
from math import prod

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

**Desafio 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características EMG além das apresentadas, citando as fontes de tais características.


## Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

## Implementação do vetor

In [4]:
chunks_time

array([[[[ 2.28618042e-05, -3.00415981e-05, -8.08431999e-05, ...,
           4.13280601e-05, -6.38167767e-06, -3.64780109e-05],
         [ 8.54354451e-06,  1.53637430e-05,  1.55777146e-05, ...,
           3.11244567e-04,  3.56579821e-04,  4.03581015e-04],
         [-1.13777478e-03, -1.04862370e-03, -8.43558373e-04, ...,
          -2.77048566e-05,  5.68830979e-06,  4.60767981e-05],
         ...,
         [-1.55637889e-04, -2.38322608e-04, -2.90895295e-04, ...,
          -1.76001678e-05, -4.85361481e-05, -9.37907915e-05],
         [-2.07960164e-04, -1.85637713e-04, -1.55034789e-04, ...,
          -4.85049608e-04, -4.64818252e-04, -3.99177132e-04],
         [-1.80405309e-04, -1.40005542e-04, -7.28989786e-05, ...,
          -8.93918526e-05, -9.92372173e-05, -8.51731769e-05]],

        [[ 4.22820844e-06,  4.63978392e-06,  1.05614328e-05, ...,
          -3.22511695e-05, -3.17068388e-05, -2.68949692e-05],
         [ 3.64485407e-05,  3.39607344e-05,  1.81930430e-05, ...,
           1.35336580e

In [3]:
final_data = list()
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))

final = np.array(final_data)
final.shape

(4, 60, 2, 41)

É necessário que seja reordenado as dimensões do vetor de características, pois cada característica (de cada canal), deve corresponder à última dimensão do vetor. Por fim, as outras dimensões são concatenadas para o número de amostras.

In [4]:
data = final.transpose(1, 3, 2, 0)
X = data.reshape(data.shape[0]*data.shape[1], data.shape[2]*data.shape[3])
X.shape

(2460, 8)

## Seleção de características

Nesta etapa, são selecionadas as características que mais afetam positivamente no resultado final da classificação. Vamos estudar os métodos de seleção de características nesta [página do projeto sklearn](https://scikit-learn.org/stable/modules/feature_selection.html).

**Desafio 3**: mostrar o resultado para os dados de trabalho, para os seguintes métodos se leção de características:
- VarianceThreshold
- Univariate feature selection
    - escolha o que mais for "interessante": `SelectKBest`, `SelectPercentile` e `GenericUnivariateSelect`
- Recursive feature elimination